# Tensor contractions

In [16]:
import numpy as np

# Index dimensions
Di = 30
Dj = 50
Dk = 20
Dl = 50
Dm = 20

A = np.random.rand(Di, Dj, Dk) # A_ijk
B = np.random.rand(Dj, Dl) # B_jl
C = np.random.rand(Dk, Dm) # C_km

In [17]:
# In this example we contract the tensor network as a series of pair wise contractions:
# (1) A * C -> AC
# (2) AC * B -> ACB

A_ij_k = np.reshape(A, [Di*Dj, Dk]) # First reshape A into a matrix
AC_ij_m = A_ij_k @ C # Contract A with C (matrix multiplication)
AC_i_j_m = np.reshape(AC_ij_m, [Di, Dj, Dm]) # Reshape resulting AC back into a tensor
AC_i_m_j = np.transpose(AC_i_j_m, [0, 2, 1]) # Permute the indices
AC_im_j = np.reshape(AC_i_m_j, [Di*Dm, Dj]) # Reshape AC into a matrix, ready to be contracted with B
ACB_im_l = AC_im_j @ B # Contract AC with B (matrix multiplication)
ACB_i_m_l = np.reshape(ACB_im_l, [Di, Dm, Dl]) # Reshape ACB back into a tensor
ACB_i_l_m = np.transpose(ACB_i_m_l, [0, 2, 1]) # Permute indices of ACB to the desired order

## Einsum

In [18]:
D = np.einsum('ijk,jl,km->ilm', A, B, C) # D_ilm

In [19]:
np.allclose(D, ACB_i_l_m)

True

In [20]:
D.shape

(30, 50, 20)

# Contraction Order

In [21]:
# Use einsum_path to calculate optimal contraction order
path_info = np.einsum_path('ijk,jl,km->ilm', A, B, C)
print(path_info[0])

['einsum_path', (0, 1), (0, 1)]


In [22]:
print(path_info[1])

  Complete contraction:  ijk,jl,km->ilm
         Naive scaling:  5
     Optimized scaling:  4
      Naive FLOP count:  9.000e+07
  Optimized FLOP count:  4.200e+06
   Theoretical speedup:  21.429
  Largest intermediate:  3.000e+04 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 jl,ijk->kil                              km,kil->ilm
   4                 kil,km->ilm                                 ilm->ilm


In [23]:
%%timeit
D = np.einsum('ijk,jl,km->ilm', A, B, C) # D_ilm

33 ms ± 496 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%%timeit
D = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=False) # D_ilm

33.3 ms ± 876 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit
D = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=['einsum_path', (1, 2), (0, 1)]) # D_ilm

2.95 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%%timeit
D = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=['einsum_path', (0, 1), (0, 1)]) # D_ilm

329 µs ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [27]:
%%timeit
D = np.einsum('ijk,jl,km->ilm', A, B, C, optimize=['einsum_path', (0, 2), (0, 1)]) # D_ilm

146 µs ± 8.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
